In [95]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import ast
from datetime import datetime
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer,IterativeImputer

from sklearn.linear_model import LinearRegression


In [96]:
#Read in csv file
train = pd.read_csv('../Datasets/train.csv')
test = pd.read_csv('../Datasets/test.csv')

In [97]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [86]:
train.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue'],
      dtype='object')

In [94]:
[(d['cast_id'],d['name'],(d['gender'])) for d in ast.literal_eval(train['cast'].iloc[12])]

[(6, 'Kevin Spacey', 2),
 (3, 'Annette Bening', 1),
 (5, 'Thora Birch', 1),
 (7, 'Wes Bentley', 2),
 (8, 'Mena Suvari', 1),
 (9, 'Chris Cooper', 2),
 (4, 'Scott Bakula', 2),
 (10, 'Peter Gallagher', 2),
 (11, 'Allison Janney', 1),
 (12, 'Sam Robards', 2),
 (13, 'Barry Del Sherman', 2),
 (29, 'John Cho', 2),
 (117, 'Hal Fort Atkinson', 2),
 (32, 'Kent Faulcon', 2),
 (25, 'Ara Celi', 1),
 (31, 'Sue Casey', 1),
 (33, 'Brenda Wehle', 1),
 (34, 'Lisa Cloud', 1),
 (36, 'Amber Smith', 1),
 (37, 'Joel McCrary', 2),
 (38, 'Marissa Jaret Winokur', 1),
 (39, 'Dennis Anderson', 0),
 (40, 'Matthew Kimbrough', 2),
 (26, 'Erin Cathryn Strubbe', 0),
 (35, 'Alison Faulk', 1),
 (42, 'Krista Goodsitt', 0),
 (43, 'Lily Houtkin', 1),
 (44, 'Carolina Lancaster', 1),
 (45, 'Mona Leah', 1),
 (46, 'Chekesha Van Putten', 1),
 (47, 'Emily Zachary', 1),
 (56, 'Nancy Anderson', 1),
 (48, 'Reshma Gajjar', 1),
 (49, 'Stephanie Rizzo', 1),
 (50, 'Heather Joy Sher', 1),
 (51, 'Chelsea Hertford', 1),
 (52, 'Elaine Corr

In [5]:
train = train.drop(columns = ['belongs_to_collection','homepage', 'imdb_id',
                              'spoken_languages','original_title','overview',
                              'poster_path','status','tagline', 'title',
                              'Keywords','cast', 'crew'])
test = test.drop(columns = ['belongs_to_collection','homepage', 'imdb_id',
                              'spoken_languages','original_title','overview',
                              'poster_path','status','tagline', 'title',
                              'Keywords','cast', 'crew'])

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                2993 non-null   object 
 3   original_language     3000 non-null   object 
 4   popularity            3000 non-null   float64
 5   production_companies  2844 non-null   object 
 6   production_countries  2945 non-null   object 
 7   release_date          3000 non-null   object 
 8   runtime               2998 non-null   float64
 9   revenue               3000 non-null   int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 234.5+ KB


# genres, production_countries, spoken_languages

##### Note for ARjun. You need to deal with missing values before you can run the following functions as is. my suggestion is that u change the functionality of the functions such that they work despite the missing values and also think of the outputs because those are what you will be dummying.

[heres lab 2.07 where you use ohe and make_column_transformer()](https://git.generalassemb.ly/kalymaan/lab-2.07-clustering/blob/main/starter-code.ipynb)

#### converting list of dictionaries which are written as strings into actual lists

In [7]:
big_dict = {'genres': 'name', 
 'production_companies': 'name', 
 'production_countries': 'name'
 #'spoken_languages': 'iso_639_1' # 'name' may be used... compare iso to original language, if same we drop
 # 'Keywords': 'name' 
 #'cast': ['gender','order'], # take the 'gender' from 'order' = 0 ******************
}



##### This is a function for the loop below it

In [8]:
def feature_extractor(l, key_we_want):
    # Input: l is the list we want to extract from.
    #        key_we_want is the key we want lol
    
    return [d[key_we_want] for d in l]

#### This is where we tutrn those crazy columns into things we can work with

In [9]:
for col_name, info_we_want in big_dict.items():
    
    
    
    train[col_name] = train[col_name][train[col_name].notna()].apply(ast.literal_eval) 
    test[col_name] = test[col_name][test[col_name].notna()].apply(ast.literal_eval) 
    
    train[col_name] = train[col_name][train[col_name].notna()].apply(lambda x: feature_extractor(x, info_we_want))
    test[col_name] =test[col_name][test[col_name].notna()].apply(lambda x: feature_extractor(x, info_we_want))
    
    
    

In [10]:
train.head()

,id,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,revenue
0,1,14000000,[Comedy],en,6.575393,"[Paramount Pictures, United Artists, Metro-Gol...",[United States of America],2/20/15,93.0,12314651
1,2,40000000,"[Comedy, Drama, Family, Romance]",en,8.248895,[Walt Disney Pictures],[United States of America],8/6/04,113.0,95149435
2,3,3300000,[Drama],en,64.299990,"[Bold Films, Blumhouse Productions, Right of W...",[United States of America],10/10/14,105.0,13092000
3,4,1200000,"[Thriller, Drama]",hi,3.174936,NaN,[India],3/9/12,122.0,16000000
4,5,0,"[Action, Thriller]",ko,1.148070,NaN,[South Korea],2/5/09,118.0,3923970


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                2993 non-null   object 
 3   original_language     3000 non-null   object 
 4   popularity            3000 non-null   float64
 5   production_companies  2844 non-null   object 
 6   production_countries  2945 non-null   object 
 7   release_date          3000 non-null   object 
 8   runtime               2998 non-null   float64
 9   revenue               3000 non-null   int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 234.5+ KB


# ok turn releasedate into unix

In [12]:
#train['release_date'] = train['release_date'].apply(lambda x: int(datetime.strptime(x, "%m/%d/%y").timestamp()))
#test['release_date'] = test['release_date'].apply(lambda x: int(datetime.strptime(x, "%m/%d/%y").timestamp()))

# dummifying the categorical columns

Will not be dropping first in the following three categorical values because they are all multicategory(not mutually exclusive)

In [22]:
train = pd.read_csv('../Datasets/train_time.csv')
test = pd.read_csv('../Datasets/test_time.csv')

In [23]:
train.drop(columns=['Unnamed: 0'], inplace=True)
test.drop(columns=['Unnamed: 0'], inplace=True)

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    3000 non-null   int64  
 1   budget                3000 non-null   int64  
 2   genres                2993 non-null   object 
 3   original_language     3000 non-null   object 
 4   popularity            3000 non-null   float64
 5   production_companies  2844 non-null   object 
 6   production_countries  2945 non-null   object 
 7   release_date          3000 non-null   int64  
 8   runtime               2998 non-null   float64
 9   revenue               3000 non-null   int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 234.5+ KB


In [25]:
train.head()

,id,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,revenue
0,1,14000000,['Comedy'],en,6.575393,"['Paramount Pictures', 'United Artists', 'Metr...",['United States of America'],1424412000,93.0,12314651
1,2,40000000,"['Comedy', 'Drama', 'Family', 'Romance']",en,8.248895,['Walt Disney Pictures'],['United States of America'],1091768400,113.0,95149435
2,3,3300000,['Drama'],en,64.299990,"['Bold Films', 'Blumhouse Productions', 'Right...",['United States of America'],1412917200,105.0,13092000
3,4,1200000,"['Thriller', 'Drama']",hi,3.174936,NaN,['India'],1331272800,122.0,16000000
4,5,0,"['Action', 'Thriller']",ko,1.148070,NaN,['South Korea'],1233813600,118.0,3923970


HAVE TO DO AST AGAIN FOR SOME FUCKIN BULLSHIT REASON

In [28]:
for col_name, info_we_want in big_dict.items():
    
    
    
    train[col_name] = train[col_name][train[col_name].notna()].apply(ast.literal_eval) 
    test[col_name] = test[col_name][test[col_name].notna()].apply(ast.literal_eval) 

In [37]:
for i,v in  train['production_countries'][train['production_countries'].notna()].items():
    if len(v) != 1 and v!=np.nan:
        print(f'Index: {i} Value {v}')

Index: 6 Value ['United States of America', 'Canada']
Index: 18 Value ['Austria', 'Germany', 'United Kingdom']
Index: 25 Value ['Japan', 'United States of America']
Index: 27 Value ['Canada', 'Ireland']
Index: 31 Value ['Austria', 'France', 'Germany', 'Italy', 'United States of America']
Index: 33 Value ['United States of America', 'New Zealand', 'Canada', 'Israel', 'Japan']
Index: 36 Value ['Ireland', 'United Kingdom', 'United States of America']
Index: 56 Value ['France', 'Germany']
Index: 57 Value ['United Kingdom', 'United States of America']
Index: 60 Value ['Italy', 'United Kingdom', 'United States of America', 'Czech Republic', 'Germany']
Index: 67 Value ['China', 'France', 'Japan']
Index: 69 Value ['Japan', 'United States of America']
Index: 70 Value ['Germany', 'United States of America']
Index: 71 Value ['Brazil', 'Argentina', 'Chile', 'France', 'Germany', 'Peru', 'United Kingdom', 'United States of America']
Index: 75 Value ['United Kingdom', 'United States of America']
Inde

In [47]:
split_index = len(train)
combined_df = pd.concat([train,test],axis=0,ignore_index=True)

In [48]:
combined_df.head()

,id,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,revenue
0,1,14000000,[Comedy],en,6.575393,"[Paramount Pictures, United Artists, Metro-Gol...",[United States of America],1.424412e+09,93.0,12314651.0
1,2,40000000,"[Comedy, Drama, Family, Romance]",en,8.248895,[Walt Disney Pictures],[United States of America],1.091768e+09,113.0,95149435.0
2,3,3300000,[Drama],en,64.299990,"[Bold Films, Blumhouse Productions, Right of W...",[United States of America],1.412917e+09,105.0,13092000.0
3,4,1200000,"[Thriller, Drama]",hi,3.174936,NaN,[India],1.331273e+09,122.0,16000000.0
4,5,0,"[Action, Thriller]",ko,1.148070,NaN,[South Korea],1.233814e+09,118.0,3923970.0


genres

In [49]:
genres_dummies = pd.get_dummies(combined_df['genres'].explode(), prefix='genre')

genres_dummies = genres_dummies.groupby(genres_dummies.index).sum()

combined_df = pd.concat([combined_df,genres_dummies], axis=1)

combined_df.drop('genres', axis=1, inplace=True)

In [50]:
combined_df.head()

,id,budget,original_language,popularity,production_companies,production_countries,release_date,runtime,revenue,genre_Action,...,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,1,14000000,en,6.575393,"[Paramount Pictures, United Artists, Metro-Gol...",[United States of America],1.424412e+09,93.0,12314651.0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,40000000,en,8.248895,[Walt Disney Pictures],[United States of America],1.091768e+09,113.0,95149435.0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,3300000,en,64.299990,"[Bold Films, Blumhouse Productions, Right of W...",[United States of America],1.412917e+09,105.0,13092000.0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1200000,hi,3.174936,NaN,[India],1.331273e+09,122.0,16000000.0,0,...,0,0,0,0,0,0,0,1,0,0
4,5,0,ko,1.148070,NaN,[South Korea],1.233814e+09,118.0,3923970.0,1,...,0,0,0,0,0,0,0,1,0,0


production_countries

In [51]:
production_country_dummies = pd.get_dummies(combined_df['production_countries'].explode(), prefix='prod_country')

production_country_dummies = production_country_dummies.groupby(production_country_dummies.index).sum()

combined_df = pd.concat([combined_df,production_country_dummies], axis=1)

combined_df.drop('production_countries', axis=1, inplace=True)

In [52]:
combined_df.head()

,id,budget,original_language,popularity,production_companies,release_date,runtime,revenue,genre_Action,genre_Adventure,...,prod_country_Taiwan,prod_country_Thailand,prod_country_Tunisia,prod_country_Turkey,prod_country_Ukraine,prod_country_United Arab Emirates,prod_country_United Kingdom,prod_country_United States of America,prod_country_Uruguay,prod_country_Venezuela
0,1,14000000,en,6.575393,"[Paramount Pictures, United Artists, Metro-Gol...",1.424412e+09,93.0,12314651.0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,40000000,en,8.248895,[Walt Disney Pictures],1.091768e+09,113.0,95149435.0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,3300000,en,64.299990,"[Bold Films, Blumhouse Productions, Right of W...",1.412917e+09,105.0,13092000.0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,1200000,hi,3.174936,NaN,1.331273e+09,122.0,16000000.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,ko,1.148070,NaN,1.233814e+09,118.0,3923970.0,1,0,...,0,0,0,0,0,0,0,0,0,0


production_company

In [53]:
production_company_dummies = pd.get_dummies(combined_df['production_companies'].explode(), prefix='prod_company')

production_company_dummies = production_company_dummies.groupby(production_company_dummies.index).sum()

combined_df = pd.concat([combined_df,production_company_dummies], axis=1)

combined_df.drop('production_companies', axis=1, inplace=True)

In [55]:
combined_df.head()

,id,budget,original_language,popularity,release_date,runtime,revenue,genre_Action,genre_Adventure,genre_Animation,...,prod_company_Кинокомпания CTB,prod_company_Кинокомпания «Lunapark»,prod_company_Леополис,prod_company_Наше Кино,prod_company_Никола Фильм,prod_company_Реал-Дакота,prod_company_Роскинопрокат,prod_company_СТВ,prod_company_Флагман-Трейд,prod_company_Фокс
0,1,14000000,en,6.575393,1.424412e+09,93.0,12314651.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,40000000,en,8.248895,1.091768e+09,113.0,95149435.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,3300000,en,64.299990,1.412917e+09,105.0,13092000.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1200000,hi,3.174936,1.331273e+09,122.0,16000000.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,ko,1.148070,1.233814e+09,118.0,3923970.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


original_language

In [56]:
dummy = pd.get_dummies(combined_df, columns=['original_language'], drop_first=True)
dummy.head()

,id,budget,popularity,release_date,runtime,revenue,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,...,original_language_sr,original_language_sv,original_language_ta,original_language_te,original_language_th,original_language_tr,original_language_ur,original_language_vi,original_language_xx,original_language_zh
0,1,14000000,6.575393,1.424412e+09,93.0,12314651.0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
1,2,40000000,8.248895,1.091768e+09,113.0,95149435.0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
2,3,3300000,64.299990,1.412917e+09,105.0,13092000.0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,4,1200000,3.174936,1.331273e+09,122.0,16000000.0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,5,0,1.148070,1.233814e+09,118.0,3923970.0,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False


In [59]:
dummy.columns

Index(['id', 'budget', 'popularity', 'release_date', 'runtime', 'revenue',
       'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy',
       ...
       'original_language_sr', 'original_language_sv', 'original_language_ta',
       'original_language_te', 'original_language_th', 'original_language_tr',
       'original_language_ur', 'original_language_vi', 'original_language_xx',
       'original_language_zh'],
      dtype='object', length=7254)

Ok splitting the daatafram back into train and test

In [62]:

train_dummified = dummy[:split_index]
test_dummified = dummy[split_index:]

In [63]:
train_dummified.shape

(3000, 7254)

In [67]:
test_dummified.shape

(4398, 7254)

In [65]:
train.shape

(3000, 10)

In [66]:
test.shape

(4398, 9)

In [69]:
test_dummified.drop(columns = ['revenue'], inplace=True)

C:\Users\kalya\AppData\Local\Temp\ipykernel_13688\1266430125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dummified.drop(columns = ['revenue'], inplace=True)


In [70]:
test_dummified.shape

(4398, 7253)

# changing 0 values to nans in 

In [74]:
train_dummified['budget'].replace(0,np.nan, inplace=True)
test_dummified['budget'].replace(0,np.nan, inplace=True)

C:\Users\kalya\AppData\Local\Temp\ipykernel_13688\2355386272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dummified['budget'].replace(0,np.nan, inplace=True)
C:\Users\kalya\AppData\Local\Temp\ipykernel_13688\2355386272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dummified['budget'].replace(0,np.nan, inplace=True)


# saving semi cleaned data

In [75]:
train_dummified.to_csv('../datasets/BIG_TRAIN.csv',index=False)
test_dummified.to_csv('../datasets/BIG_TEST.csv',index=False)

# Train test

In [73]:
x = train_dummified.drop(columns=['id','revenue'])
y = train_dummified['revenue']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)


NameError: name 'train_test_split' is not defined

# Making a length of title column

In [ ]:
combined_df['length_of_title'] = combined_df['original_title'].str.len()


In [ ]:
combined_df.drop(columns=['original_title'], inplace=True)

In [ ]:
test_dummified = test_dummified.drop(columns=['revenue'])

In [ ]:
test_dummified.shape

In [ ]:
train_dummified.shape

hello 